In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

PATH = "/content/drive/MyDrive/KUBIG/RecommendationSystem/project"

Mounted at /content/drive


In [ ]:
df1=pd.read_csv(PATH+"/cluster1.csv")

In [ ]:
df1

,Unnamed: 0,User-ID,Book-Rating,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher,state,clusters
0,0,USER_00003,9,20-24,Wir sind Erinnerung. GedÃ?Â¤chtnis und PersÃ?Â...,Daniel L. Schacter,2000+,Rowohlt Tb.,others,1
1,1,USER_00003,7,20-24,2061. Odyssee III. Roman.,Arthur C. Clarke,2000+,Heyne,others,1
2,2,USER_00003,7,20-24,Die Jury. Roman.,John Grisham,1990-1994,Heyne,others,1
3,3,USER_00003,8,20-24,Frisch gepreÃ?Â?t.,Susanne FrÃ?Â¶hlich,2000+,Ullstein Buchverlage GmbH &amp; Co. KG / Ullst...,others,1
4,4,USER_00003,6,20-24,Hitlers Helfer.,Guido Knopp,1995-1999,Goldmann,others,1
...,...,...,...,...,...,...,...,...,...,...
143441,687942,USER_92078,9,30-34,Red Dwarf,Grant Naylor,2000+,Penguin Books Ltd,others,1
143442,687943,USER_92078,0,30-34,Valley Of The Far Side,Gary Larson,1985-1989,Andrews McMeel Publishing,others,1
143443,687944,USER_92078,0,30-34,Snow Crash,Neal Stephenson,1990-1994,Bantam,others,1
143444,687945,USER_92078,0,30-34,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000+,Scholastic,others,1


# 클러스터 1의 유저에 대해 추천 진행

# 유저-책 평점 행렬 생성

In [ ]:
import random
import statistics
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity
import operator

In [ ]:
rating_matrix = df1.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating')

In [ ]:
rating_matrix.index = rating_matrix.index.rename('user_id')

In [ ]:
rating_matrix = rating_matrix.fillna(0)
rating_matrix

Book-Title,Deceived,"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",Final Fantasy Anthology: Official Strategy Guide (Brady Games),Flight of Fancy: American Heiresses (Zebra Ballad Romance),Garfield Bigger and Better (Garfield (Numbered Paperback)),God's Little Promise Book,"Goosebumps Monster Edition 1: Welcome to Dead House, Stay Out of the Basement, and Say Cheese and Die!",LA Gallinita Roja/the Little Red Hen,Mystery Mile,"Q-Space (Star Trek The Next Generation, Book 47)",...,seaQuest 2,sed &amp; awk (A Nutshell handbook),stardust,teach yourself...C++,them (Modern Library),"wet sand, raven tracks",why I'm like this : True Stories,"Ã?aque, o, De piojos y actores ;: Ay, Carmela! (Letras hispÃ¡nicas)","Ã?Â?berall, nur nicht hier",Ã?Â?thique en toc
user_id,,,,,,,,,,,,,,,,,,,,,
USER_00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_00077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_00136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_00207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_00209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USER_91794,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_91895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USER_91905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def create_similarity_matrix(rating_matrix):
    ratings = rating_matrix.values

    similarity_matrix = cosine_similarity(ratings)

    similarity_matrix = pd.DataFrame(similarity_matrix, index=rating_matrix.index, columns=rating_matrix.index)

    return similarity_matrix

In [ ]:
similarity_matrix = create_similarity_matrix(rating_matrix)

In [ ]:
def get_recommendations(user_id, rating_matrix, similarity_matrix, top_n=5):
    user_ratings = rating_matrix.loc[user_id]
    user_similarity = similarity_matrix.loc[user_id]

    # 유사도가 가장 높은 사용자 찾기
    similar_users = user_similarity.sort_values(ascending=False).index[1:]

    recommended_books = []
    target = []
    for user in similar_users:
        other_user_ratings = rating_matrix.loc[user]
        unrated_books = other_user_ratings[other_user_ratings == 0].index
        rated_books = other_user_ratings[other_user_ratings > 0].index

        # 그 사용자가 높은 평점을 준 책을 목록에 추가
        recommended_books.append(rated_books)

        point=df1[df1['User-ID']==user_id]['Book-Title']
        point=point.tolist()
        recommended_books=recommended_books[0]
        recommended_books=recommended_books.tolist()
        for i in range(len(point)):
          if point[i] in recommended_books:
            recommended_books.remove(point[i])
        target.append(recommended_books)


        # 지정 추천 개수까지 반복
        if len(recommended_books) >= top_n:
            break

    # 상위 n개 선택
    top_books = recommended_books[:top_n]

    return top_books


#추천 목록 확인

In [ ]:
user_id = 'USER_00003'
recommendations = get_recommendations(user_id, rating_matrix, similarity_matrix, top_n=5)
print(recommendations)

['Der Klient.', 'Der Ruf des Falken.', 'Der magische Steinkreis. Das groÃ?Â?e Kompendium zur Highland- Saga.', 'Die geliehene Zeit.', 'Ferne Ufer. Der 3. Band der groÃ?Â?en Highland- Saga.']


In [ ]:
user_id = 'USER_00077'
recommendations = get_recommendations(user_id, rating_matrix, similarity_matrix, top_n=5)
print(recommendations)

['Dark Rivers of the Heart: A Novel', 'Miracle on the 17th Green: A Novel', 'The Beach House', 'The Goal: A Process of Ongoing Improvement', 'The Race']


In [ ]:
user_id = 'USER_00136'
recommendations = get_recommendations(user_id, rating_matrix, similarity_matrix, top_n=5)
print(recommendations)

['A Deeper Shade of Grace', 'A Few Minutes with Andy Rooney', 'Another Homecoming', 'Boy Meets Girl', 'China Trade (A Bill Smith/Lydia Chin Novel)']
